Data Collection

In [ ]:
import ccxt
import pandas as pd
from datetime import datetime

def fetch_data(symbol='BTC/USDT', time_frame='1h', from_ts_str='2021-01-01 00:00:00'):
    # Initialize the Binance exchange API
    binance = ccxt.binance()

    from_ts = int(datetime.timestamp(datetime.strptime(from_ts_str, '%Y-%m-%d %H:%M:%S')) * 1000)

    # Fetch new data from the exchange
    new_ohlcv = binance.fetch_ohlcv(symbol, time_frame, since=from_ts, limit=1000)
    
    # Initialize the DataFrame with the first batch of data
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df = pd.DataFrame(new_ohlcv, columns=columns)

    batch_number = 1
    while len(new_ohlcv) == 1000:
        print(f"Fetching batch {batch_number}...")
        from_ts = new_ohlcv[-1][0] + 1  # add 1 millisecond to avoid duplicates
        new_ohlcv = binance.fetch_ohlcv(symbol, time_frame, since=from_ts, limit=1000)
        
        # Append new data to the DataFrame
        df = pd.concat([df, pd.DataFrame(new_ohlcv, columns=columns)], ignore_index=True)

        batch_number += 1

    print("Data fetching completed!")
    return df


Data Preprocessing & Feature Engineering

In [ ]:
import talib
from sklearn.preprocessing import MinMaxScaler

def preprocess_data(df):
    df['rsi'] = talib.RSI(df['close'].values, timeperiod=14)
    df.dropna(inplace=True)
    scaler_price = MinMaxScaler(feature_range=(0, 1))
    df['scaled_close'] = scaler_price.fit_transform(
        df['close'].values.reshape(-1, 1))
    scaler_rsi = MinMaxScaler(feature_range=(0, 1))
    df['scaled_rsi'] = scaler_rsi.fit_transform(
        df['rsi'].values.reshape(-1, 1))
    return df, scaler_price


Prepare the LSTM dataset

In [ ]:
import numpy as np

def create_dataset(df):
    X, y = [], []
    for i in range(60, len(df)):
        X.append(np.column_stack(
            (df.scaled_close.values[i-60:i], df.scaled_rsi.values[i-60:i])))
        y.append(df.scaled_close.values[i])
    return np.array(X), np.array(y)

Build the LSTM model

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

def create_model():
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(60, 2)))
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Run the logic flow

In [ ]:
import os
from keras.models import load_model

df = fetch_data()
df, scaler_price = preprocess_data(df)
X, y = create_dataset(df)

model_path = "bitcoin_prediction_model.keras"

if os.path.exists(model_path):
    model = load_model(model_path)
    print("Model loaded successfully!")
else:
    model = create_model()
    print("Created a new model.")
model.fit(X, y, epochs=10, batch_size=64)

model.save(model_path)
print(f"Model updated and saved to {model_path}")

last_60 = df[-60:]
next_input = np.column_stack((last_60.scaled_close.values, last_60.scaled_rsi.values))
next_input = np.reshape(next_input, (1, 60, 2))
prediction = model.predict(next_input)

predicted_price = scaler_price.inverse_transform(prediction)
print(f"Predicted BTC Price for next 1h: ${predicted_price[0][0]}")